# Photoreceptor segmentation using [napari](https://napari.org/) and [cellpose](https://cellpose-napari.readthedocs.io/en/latest/index.html)

### Basic usage:
```python
viewer = napari.Viewer()
nbscreenshot(viewer)
viewer.close()
```

example notebooks [here](https://github.com/sofroniewn/napari-training-course/blob/master/lessons/)

In [2]:
# !mamba install -c conda-forge devbio-napari --yes
# !jupyter labextension list
# !mamba install -c conda-forge cellpose-napari --yes
# from cellpose import models

ModuleNotFoundError: No module named 'cellpose'

In [1]:
# image analysis
import napari
from magicgui import magicgui
from enum import Enum
import cellpose_napari
import cellpose
from cellpose import models
from tifffile import imread
import napari_nikon_nd2
from skimage.io import imread
from magicgui import magicgui
# python basics and plotting
import numpy as np
import pandas as pd
import math
from scipy import ndimage
from scipy.stats import kruskal
from scipy.stats import zscore
from scipy.stats import sem
import scikit_posthocs
import scipy.spatial
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import svgutils
from svgutils.compose import *
# system interaction
import os.path
import time, sys, warnings, re
from tqdm.notebook import tqdm, trange
import importlib
import ipywidgets
from IPython.display import display
# lab brews
from plotParams import *
from lloyd import *


def getFileList(dirData,gene,fileNameMatch,addDetails=''):
   fileList = sorted([f for f in os.listdir(dirData) if not f.startswith('.')]) # get all the files in the data directory (excluding system files) and sort them alphabatically
   fileList = ([f for f in fileList if gene in f]) # select files that match the gene of interest
   fileList = ([f for f in fileList if any([s in f for s in fileNameMatch])]) # only keep relevant files
   if (addDetails=='path'):
      fileList = (['# filePath = \'' + f for f in fileList]) # add text before each file
      fileList = list(map(lambda st: str.replace(st, '.nd2', '\'; gene = \'' + gene + '\'; '), fileList)) #remove file extension (.nd2) and add text after each file
   elif (addDetails=='list'):
      fileList = (['\'' + f for f in fileList]) # add text before each file
      fileList = list(map(lambda st: str.replace(st, '.nd2', '\','), fileList)) #remove file extension (.nd2) and add text after each file
   else:
      fileList = list(map(lambda st: str.replace(st, '.nd2', ''), fileList)) #remove file extension (.nd2) and add text after each file
   return fileList


pBGMenu = ipywidgets.Dropdown(options=['Light', 'Dark'],value='Dark',disabled=False, layout=ipywidgets.Layout(width='20%', height='40px'))
pBGLabel = ipywidgets.widgets.Label('Select plotting style', layout=ipywidgets.Layout(width='20%', height='30px'))
pBG = ipywidgets.widgets.VBox([pBGLabel,pBGMenu])
pBG

In [2]:
applyPlotStyle(pBGMenu.value)
viewer = napari.Viewer()

Plotting style is Dark


In [61]:
# viewer = napari.Viewer()

In [60]:
# viewer.close()

***
# Index <a id='Index'></a>
***
- [define directories](#dirDef)
- [Extraction of z planes for analysis](#zExtract)
- [Segmentation with cellpose](#cellSeg)
- [Manual correction of segmentation](#manualCuration)
- [Create thumbnails _WIP_](#thumbnails)
- [Quantification](#quantification)

***
## Define directories<a id='dirDefdirDef'>∮</a>
***
[Back to Index](#Index)

In [3]:
# define where all the imaging data is stored
dirData = 'D:/Imaging/A1R/20220128_tbx2bF3_nr2e3F0_immunos/'
# define where all the analysis files should be stored
# subdirectories for each gene and transgenic lines need to be manually made first
dirAnalysis = 'G:/Shared drives/vldImaging/Analysis/F3tbx2bF0nr2e3_Immunos/xOG_sws1Cy3_rhoCy5/'

In [4]:
gene = 'uninj' # usually 'wt' or gene of interest (e.g. 'tbx2a', 'lhx1a')
# gene = 'lhx1a' # usually 'wt' or gene of interest (e.g. 'tbx2a', 'lhx1a')
fileNameMatch = ['002','004'] # characters in filenames that define zoom-in files

# get a list with all the relevant files
print(*getFileList(dirData,gene,fileNameMatch,addDetails='path'), sep = "\n") #display the file list for copy and paste below

# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L13_002'; gene = 'uninj'; 
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L14_002'; gene = 'uninj'; 
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L14_004'; gene = 'uninj'; 
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L15_002'; gene = 'uninj'; 
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L15_004'; gene = 'uninj'; 
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L16_002'; gene = 'uninj'; 
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L17_002'; gene = 'uninj'; 
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L17_004'; gene = 'uninj'; 
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L18_002'; gene = 'uninj'; 
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L18_004'; gene = 'uninj'; 
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L19_002'; gene = 'uninj'; 
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L20_002'; gene = 'uninj'; 
# fi

***
## Extract layers from z-stacks<a id='zExtract'>∮</a>
> Open z-stack &rarr; Pick limits in z for Maximum-Intensity Projection (_MIP_) &rarr; save MIPs &rarr; Ensure image was located in central retina
***
[Back to Index](#Index)

In [13]:
# clear viewer if needed
viewer.layers.select_all()
viewer.layers.remove_selected()

In [7]:
# clear viewer before any loading
viewer.layers.select_all()
viewer.layers.remove_selected()

# open files one by one (by commenting and uncommenting each one)

#Uninjected
filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L13_002'; gene = 'uninj'; zlims = [18,20];#Only GFP channel
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L14_002'; gene = 'uninj'; zlims = [24,26];#same here
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L14_004'; gene = 'uninj'; zlims = [32,34];  #same here whole eye mip not taken
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L15_002'; gene = 'uninj'; zlims = [15,17]; #only two channels
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L15_004'; gene = 'uninj'; zlims = [31,33];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L16_002'; gene = 'uninj'; zlims = [40,42]; #all channels exclude optic nerve included
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L17_002'; gene = 'uninj'; zlims = [23,25];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L17_004'; gene = 'uninj'; zlims = [25,27];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L18_002'; gene = 'uninj'; zlims = [26,28]
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L18_004'; gene = 'uninj'; zlims = [25,27];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L19_002'; gene = 'uninj'; zlims = [39,41];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L20_002'; gene = 'uninj'; zlims = [39,41];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L21_002'; gene = 'uninj'; zlims = [24,26]; 
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L22_002'; gene = 'uninj'; zlims = [22,24];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L23_002'; gene = 'uninj'; lims = [35,37]
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L24_002'; gene = 'uninj'; zlims = [30,32];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L24_004'; gene = 'uninj'; zlims = [32,34];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L25_002'; gene = 'uninj'; zlims = [27,29]
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L26_002'; gene = 'uninj'; zlims = [34,36];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L26_004'; gene = 'uninj'; zlims = [33,35]
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L27_002'; gene = 'uninj'; zlims = [35,37];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L28_002'; gene = 'uninj'; zlims = [30,32];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L28_004'; gene = 'uninj'; zlims = [29,31];

#Injected
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L25_002'; gene = 'nr2e3F0'; zlims = [23,25]
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L25_004'; gene = 'nr2e3F0'; zlims = [26,28];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L26_002'; gene = 'nr2e3F0'; zlims = [23,25];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L26_004'; gene = 'nr2e3F0'; zlims = [43,45];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L27_002'; gene = 'nr2e3F0'; zlims = [23,25];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L27_004'; gene = 'nr2e3F0'; zlims = [29,31]
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L28_002'; gene = 'nr2e3F0'; zlims = [28,30]; 
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L28_004'; gene = 'nr2e3F0'; zlims = [37,39]
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L29_002'; gene = 'nr2e3F0'; zlims = [28,30]; #did not save 001 mip
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L29_004'; gene = 'nr2e3F0'; zlims = [35,37];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L30_002'; gene = 'nr2e3F0'; zlims = [31,33];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L30_002bis'; gene = 'nr2e3F0'; zlims = [29,31]; #no whole eye mip
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L30_004'; gene = 'nr2e3F0'; zlims = [27,29];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L32_002'; gene = 'nr2e3F0'; zlims = [25,27]
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L32_004'; gene = 'nr2e3F0'; zlims = [28,30];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L33_002'; gene = 'nr2e3F0'; zlims = [22,24]
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L33_004'; gene = 'nr2e3F0'; zlims = [34,36];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L34_002'; gene = 'nr2e3F0'; zlims = [38,40];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L34_004'; gene = 'nr2e3F0'; zlims = [39,41]; #exclude optic nerve included
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L35_002'; gene = 'nr2e3F0';zlims = [26,28]; 
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L36_002'; gene = 'nr2e3F0'; zlims = [28,30]; 
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L36_004'; gene = 'nr2e3F0'; zlims = [25,27];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L37_002'; gene = 'nr2e3F0'; zlims = [30,32];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L37_002bis'; gene = 'nr2e3F0'; zlims = [33,35];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L37_004'; gene = 'nr2e3F0'; zlims = [25,27]; 
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L38_002'; gene = 'nr2e3F0'; zlims = [36,38]; 
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L38_004'; gene = 'nr2e3F0'; zlims = [37,39]; #no whole eye mip
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L39_002'; gene = 'nr2e3F0'; zlims = [38,40];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L40_002'; gene = 'nr2e3F0'; zlims = [25,27]; 
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L40_004'; gene = 'nr2e3F0'; zlims = [24,26]
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L41_002'; gene = 'nr2e3F0'; zlims = [17,19];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L41_004'; gene = 'nr2e3F0'; zlims = [29,31];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L42_002'; gene = 'nr2e3F0'; zlims = [25,27];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L42_004'; gene = 'nr2e3F0'; zlims = [27,29]
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L43_002'; gene = 'nr2e3F0'; zlims = [35,37]
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L43_002_b'; gene = 'nr2e3F0'; zlims = [35,37]; #no map of whole retina
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L43_004'; gene = 'nr2e3F0'; zlims = [26,28]; 
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L44_002'; gene = 'nr2e3F0'; zlims = [30,32]; #no mip taken
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L44_004'; gene = 'nr2e3F0'; zlims = [29,31]; #no whole mip
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L45_002'; gene = 'nr2e3F0'; zlims = [22,24]; 
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L45_004'; gene = 'nr2e3F0'; zlims = [19,21];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L46_002'; gene = 'nr2e3F0';zlims = [21,23];  
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L46_004'; gene = 'nr2e3F0'; zlims = [28,30];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L47_002'; gene = 'nr2e3F0'; zlims = [24,26]; 
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L48_002'; gene = 'nr2e3F0'; zlims = [27,29]; #no whole eye mip taken
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L49_002'; gene = 'nr2e3F0'; zlims = [21,23];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L49_004'; gene = 'nr2e3F0'; zlims = [21,23] #No L49 003 mip
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L50_002'; gene = 'nr2e3F0'; zlims = [20,22];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L50_004'; gene = 'nr2e3F0'; zlims = [27,29]; #No L509 003 mip
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L51_002'; gene = 'nr2e3F0'; zlims = [21,24]; #4 rods outside central retina
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L51_004'; gene = 'nr2e3F0'; zlims = [17,19];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L52_002'; gene = 'nr2e3F0'; zlims = [25,27];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L52_004'; gene = 'nr2e3F0'; zlims = [29,31]
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L56_002'; gene = 'nr2e3F0'; zlims = [24,36];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L56_002b'; gene = 'nr2e3F0'; zlims = [23,25]; #same as above file
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L56_004'; gene = 'nr2e3F0'; zlims = [26,28];

layerNames = ['N','R']

# create subdirectory for analysis output
dirOut = dirAnalysis + gene + '/' + filePath + '/'
if (os.path.isdir(dirOut)==False):
   os.mkdir(dirOut)
   print('Created output directory')

print('Canvas cleared. Loading data...\n\n')

# load whole stack to figure out best z-planes for cellpose
viewer.open((dirData+filePath + '.nd2'), plugin='napari-nikon-nd2')
viewer.layers.remove(viewer.layers[len(viewer.layers)-1]) # remove transmitted detector image
viewer.layers.select_next();
if len(viewer.layers)==2: #no DAPI
   nChannels = 2
   viewer.layers[0].colormap = 'magenta'
   viewer.layers[0].name = layerNames[0]
   viewer.layers[1].colormap = 'green'
   viewer.layers[1].name = layerNames[1]
elif len(viewer.layers)==3: #DAPI
   nChannels = 3
# im not sure this WORKS!!!!!
   viewer.layers[0].colormap = 'magenta'
   viewer.layers[0].name = layerNames[0]
   viewer.layers[1].colormap = 'green'
   viewer.layers[1].name = layerNames[1]
   viewer.layers[2].colormap = 'bop orange'
   viewer.layers[2].name = layerNames[2]
    
   # viewer.layers[0].colormap = 'gray'
   # viewer.layers[0].name = 'N'
   # viewer.layers[0].opacity = 0.99
   # viewer.layers[1].colormap = 'green'
   # viewer.layers[1].name = layerNames[0]
   # viewer.layers[2].colormap = 'magenta'
   # viewer.layers[2].name = layerNames[1]
    # layerNames.append('N')


elif len(viewer.layers)==4: #DAPI
   nChannels = 4
   viewer.layers[0].colormap = 'gray'
   viewer.layers[0].name = 'N'
   viewer.layers[0].opacity = 0.99
   viewer.layers[1].colormap = 'green'
   viewer.layers[1].name = layerNames[0]
   viewer.layers[2].colormap = 'magenta'
   viewer.layers[2].name = layerNames[1]
   viewer.layers[3].colormap = 'bop orange'
   viewer.layers[3].name = layerNames[2]
   layerNames.append('N')

print('Images loaded!')

Canvas cleared. Loading data...


Images loaded!


In [10]:
# make mips (and remove any previous ones) by defining the z-limits to use
zlims = [23,25]; # for photoreceptor/nuclei counts
# zlims2 = [17,19]; # for immuno

if len(viewer.layers)>nChannels:
    for l in viewer.layers[nChannels:]:
      viewer.layers.remove(l)

# for l in viewer.layers[0:nChannels]:
for l in [viewer.layers['R'],viewer.layers['N']]:
    l.visible = False
    viewer.add_image(l.data[zlims[0]:zlims[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")

# for l in [viewer.layers['Rho']]:
#     l.visible = False
#     viewer.add_image(l.data[zlims2[0]:zlims2[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")

In [11]:
# SAVE in folder for batch cellpose analysis (when mips look good)
for name in layerNames:
   l = viewer.layers[name + '_mip']; l.save(dirOut + l.name + '.tiff')
print('MIP layers saved for ' + filePath)

MIP layers saved for 20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L13_002


In [14]:
### Save Image from Viewer and Adjust Brightness
# savePNGfromNapari(viewer,'G:/Shared drives/vldImaging/Analysis/F4tbx2a/KO2bFOnr2e3')

### Establish landmarks in whole-eye MIP
- Use eyeMarks layer to mark:
    - [ ] Landmark #1: Center of FOV (where zoom in is centered)
    - [ ] Landmark #2: Center of Optic Nerve Head 
    - [ ] Landmark #3: Dorsal edge of eye
    - [ ] Landmark #4: Ventral edge of eye
    - [ ] Landmark #5: Temporal edge of eye (strike zone side)
    - [ ] Landmark #6: Nasal edge of eye
- Decide if stack should be excluded (and stop analysis here and make note in csv summary file)

In [9]:
nMIP = len(layerNames)
for name in layerNames:
   lName = name + "_mip"
   viewer.layers[lName].scale = [1.5/8,1.5/8];
   viewer.layers[lName].translate = [0,+(512-1024*1.5/(8*2))]

# Load whole-eye MIP to establish landmarks
print('Loading eye mip...')
if str.endswith(filePath,'002'):
    mipPath = filePath[:-1] + '1-MaxIP';
elif str.endswith(filePath,'004'):
    mipPath = filePath[:-1] + '3-MaxIP';
elif str.endswith(filePath,'b'):
    mipPath = filePath[:-1] + 'a_MaxIP';
elif str.endswith(filePath,'d'):
    mipPath = filePath[:-1] + 'c_MaxIP';
viewer.open((dirData+mipPath + '.nd2'), plugin='napari-nikon-nd2', blending='additive')

viewer.layers.remove(viewer.layers[len(viewer.layers)-1]) # remove transmitted detector image
viewer.layers.select_next();
eyeChannels = 0
if len(viewer.layers)==2+nMIP+nChannels: # no DAPI, GFP and RFP
   eyeChannels = 2
   viewer.layers[nMIP+nChannels].colormap = 'green'
   viewer.layers[nMIP+nChannels].name = 'eyeGreen'
   viewer.layers[nMIP+nChannels+1].colormap = 'magenta'
   viewer.layers[nMIP+nChannels+1].name = 'eyeRed'
elif len(viewer.layers)==3+nMIP+nChannels: # leave DIC
   eyeChannels = 3
   viewer.layers[nMIP+nChannels+0].colormap = 'green'
   viewer.layers[nMIP+nChannels+0].name = 'eyeGreen'
   viewer.layers[nMIP+nChannels+1].colormap = 'magenta'
   viewer.layers[nMIP+nChannels+1].name = 'eyeRed'
   viewer.layers[nMIP+nChannels+2].colormap = 'gray'
   viewer.layers[nMIP+nChannels+2].name = 'eyeBlue'
   viewer.layers[nMIP+nChannels+2].opacity = 0.33
elif len(viewer.layers)==4+nMIP+nChannels: #DIC + DAPI
   eyeChannels = 4
   viewer.layers[nMIP+nChannels+0].colormap = 'gray'
   viewer.layers[nMIP+nChannels+0].name = 'eyeBlue'
   viewer.layers[nMIP+nChannels+1].colormap = 'green'
   viewer.layers[nMIP+nChannels+1].name = 'eyeGreen'
   viewer.layers[nMIP+nChannels+2].colormap = 'magenta'
   viewer.layers[nMIP+nChannels+2].name = 'eyeRed'
   viewer.layers[nMIP+nChannels+3].colormap = 'bop orange'
   viewer.layers[nMIP+nChannels+3].name = 'eyePNA'
   viewer.layers[nMIP+nChannels+3].opacity = 0.33
print(eyeChannels)
print('Loaded: ' + filePath)
eyeMarks = viewer.add_points(size=20, name = 'eyeMarks', symbol='cross', face_color='#ffffff80')
viewer.layers.select_next();

# Save eye landmarks
eyeMarksButton = ipywidgets.Button(description='Save eyeMarks', layout=ipywidgets.Layout(width='200px', height='100px'))
out = ipywidgets.Output()
def eyeMClick(_):
   with out:
      l = viewer.layers['eyeMarks']; l.save(dirOut + l.name + '.csv');
      print('eyeMarks layers saved for ' + filePath)
   # clear viewer
   viewer.layers.select_all()
   viewer.layers.remove_selected()
   eyeMarksButton.disabled = True # to prevent double clicking warnings. All sales are final!

eyeMarksButton.on_click(eyeMClick)

print('Make 6 landmark points then click button to save')

ipywidgets.VBox([eyeMarksButton,out])

NameError: name 'layerNames' is not defined

### [Go back and run next one &uarr;](#zExtract)

***
## Segmentation using cellpose<a id='cellSeg'>∮</a>
> Define list of files to segment &rarr; Define cellpose parameters &rarr; Run cellpose in each image
***
[Back to Index](#Index)

### Running cellpose as batch analysis
Usually for:
- Rods: diameter = 20; flow_threshold = 0.5 and mask_threshold = 0.0
- Cones: diameter = 40; flow_threshold = 0.5 and mask_threshold = 0.0
- Horizontal cells: diameter = 100; flow_threshold = 0.6 and mask_threshold = 0.0

> Using a higher _flow_threshold_ (i.e. 0.6 instead of 0.4) will provide more ROIs

In [127]:
# get file list again
gene = 'nr2e3F0'
print(*getFileList(dirData,gene,fileNameMatch,addDetails='list'), sep = "\n") #display the file list for copy and paste below

'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L25_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L25_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L26_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L26_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L27_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L27_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L28_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L28_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L29_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L29_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L30_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L30_002bis',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L30_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L32_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L32_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L33_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L33_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L34_002',
'202201

In [38]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()

# collect file paths for wild-type
gene_wt = 'uninj'
filePaths_wt = [
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L13_002',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L14_002',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L14_004',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L15_002',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L15_004',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L16_002',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L17_002',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L17_004',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L18_002',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L18_004',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L19_002',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L20_002',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L21_002',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L22_002',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L23_002',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L24_002',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L24_004',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L25_002',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L26_002',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L26_004',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L27_002',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L28_002',
# '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L28_004',
         ]
# collect file paths for mutants
gene_F0 = 'nr2e3F0'
filePaths_F0 = [
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L25_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L25_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L26_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L26_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L27_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L27_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L28_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L28_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L29_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L29_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L30_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L30_002bis',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L30_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L32_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L32_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L33_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L33_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L34_002',
# '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L34_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L35_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L36_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L36_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L37_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L37_002bis',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L37_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L38_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L38_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L39_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L40_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L40_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L41_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L41_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L42_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L42_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L43_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L43_002_b',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L43_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L44_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L44_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L45_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L45_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L46_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L46_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L47_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L48_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L49_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L49_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L50_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L50_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L51_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L51_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L52_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L52_004',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L56_002',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L56_002b',
'20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L56_004',
         ]


# define cellpose parameters
cpParams = {
    'model' : 'cyto2', # default is 'cyto' or 'cyto2'
    'net_avg' : True,
    'channels' : [0,0], #single channel without nucleus info
    'diameterR' : 25,
    'diameterC' : 40,
    'diameterH' : 100,
    'diameterOS' : 20, # Carinna vows to check this number!!!!
    'flow_threshold' : 0.5,
    'cellprob_threshold' : 0.0
}


# define model to use (e.g. 'cyto2')
model = models.Cellpose(gpu=False, model_type=cpParams['model'])

#### Run in loop to track progress and automatically save results
> In Jan 2022: ~4 minutes per 1024 x 1024 image (4 nets). 

In [41]:
print('Starting analysis for '+ gene_wt + ':')
with tqdm(total=len(filePaths_wt)*len(layerNames), file=sys.stdout) as progBar:
    for fp in filePaths_wt:
        print('\t' + fp)
        for l in layerNames:
            print('\t\t ' + l)
            startTime = time.time()
            imgPath = dirAnalysis + gene_wt + '/' + fp + '/' + l + '_mip.tiff'
            if (os.path.isfile(imgPath)):
                img = cellpose.io.imread(imgPath)
                if l=='R':
                    diameterCell = cpParams['diameterR']
                elif l=='H':
                    diameterCell = cpParams['diameterH']
                else:
                    diameterCell = cpParams['diameterC']
                masks, flows, styles, diams = model.eval(img, 
                                                    diameter=diameterCell, channels=cpParams['channels'],
                                                    do_3D=False, net_avg = cpParams['net_avg'], interp = True,
                                                    flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
                cellpose.io.masks_flows_to_seg(img, masks, flows, diams, imgPath, cpParams['channels']) # save results
            else:
                print('\t\t' + l + ' not found for ' + fp)
            endTime = time.time()
            print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
            progBar.update(1)

print('Finished cellpose batch analysis for '+ gene_wt)

print('Starting analysis for '+ gene_F0 + ':')
with tqdm(total=len(filePaths_F0)*len(layerNames), file=sys.stdout) as progBar:
      for fp in filePaths_F0:
        print('\t' + fp)
        for l in layerNames:
            print('\t\t ' + l)
            startTime = time.time()
            imgPath = dirAnalysis + gene_F0 + '/' + fp + '/' + l + '_mip.tiff'
            if (os.path.isfile(imgPath)):
                img = cellpose.io.imread(imgPath)
                if l=='R':
                    diameterCell = cpParams['diameterR']
                elif l=='H':
                    diameterCell = cpParams['diameterH']
                else:
                    diameterCell = cpParams['diameterC']
                masks, flows, styles, diams = model.eval(img, 
                                                    diameter=diameterCell, channels=cpParams['channels'],
                                                    do_3D=False, net_avg = cpParams['net_avg'], interp = True,
                                                    flow_threshold = cpParams['flow_threshold'], cellprob_threshold = cpParams['cellprob_threshold'])
                cellpose.io.masks_flows_to_seg(img, masks, flows, diams, imgPath, cpParams['channels']) # save results
            else:
                print('\t\t' + l + ' not found for ' + fp)
            endTime = time.time()
            print ('\t Time elapsed: {elapsedTime} s\n'.format(elapsedTime = int(endTime - startTime)))
            progBar.update(1)


print('Finished cellpose batch analysis for '+ gene_F0)

Starting analysis for uninj:


0it [00:00, ?it/s]

Finished cellpose batch analysis for uninj
Starting analysis for nr2e3F0:


  0%|          | 0/165 [00:00<?, ?it/s]

	20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L25_002
		 R
	 Time elapsed: 22 s

		 U
	 Time elapsed: 13 s

		 Rho
	 Time elapsed: 15 s

	20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L25_004
		 R
	 Time elapsed: 23 s

		 U
	 Time elapsed: 13 s

		 Rho
	 Time elapsed: 16 s

	20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L26_002
		 R
	 Time elapsed: 22 s

		 U
	 Time elapsed: 13 s

		 Rho
		Rho not found for 20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L26_002
	 Time elapsed: 0 s

	20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L26_004
		 R
	 Time elapsed: 24 s

		 U
	 Time elapsed: 13 s

		 Rho
		Rho not found for 20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L26_004
	 Time elapsed: 0 s

	20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L27_002
		 R
	 Time elapsed: 23 s

		 U
	 Time elapsed: 12 s

		 Rho
		Rho not found for 20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L27_002
	 Time elapsed: 0 s

	20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L27_004
		 R
	 Time elapsed: 24 s

		 U
	 Time elapsed: 16 s



***
## Manual correction of cellpose segmentation<a id='manualCuration'>∮</a>
> Copy file list from "Extract Layers" &rarr; Open a single file and manually fix ("_curate_") cellpose segmentation

WARNINGS:
- In tg[_mws2_:GFP], a subset of S cones are GFP+. If available, use tg[_sws2_:mCherry] to delete those labels from the M-cone segmentation
***
[Back to Index](#Index)

In [15]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()

#uninjected files
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L13_002'; gene = 'uninj'; zlims = [18,20];#Only GFP channel
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L14_002'; gene = 'uninj'; zlims = [24,26];#same here
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L14_004'; gene = 'uninj'; zlims = [32,34];  #same here whole eye mip not taken
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L15_002'; gene = 'uninj'; zlims = [15,17]; #only two channels
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L15_004'; gene = 'uninj'; zlims = [31,33]; #mips okay
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L16_002'; gene = 'uninj'; zlims = [40,42]; # mips okay, all channels exclude optic nerve included
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L17_002'; gene = 'uninj'; zlims = [23,25];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L17_004'; gene = 'uninj'; zlims = [25,27];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L18_002'; gene = 'uninj'; zlims = [26,28]
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L18_004'; gene = 'uninj'; zlims = [25,27];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L19_002'; gene = 'uninj'; zlims = [39,41];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L20_002'; gene = 'uninj'; zlims = [39,41];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L21_002'; gene = 'uninj'; zlims = [24,26]; 
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L22_002'; gene = 'uninj'; zlims = [22,24];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L23_002'; gene = 'uninj'; lims = [35,37]
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L24_002'; gene = 'uninj'; zlims = [30,32];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L24_004'; gene = 'uninj'; zlims = [32,34];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L25_002'; gene = 'uninj'; zlims = [27,29]
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L26_002'; gene = 'uninj'; zlims = [34,36];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L26_004'; gene = 'uninj'; zlims = [33,35]
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L27_002'; gene = 'uninj'; zlims = [35,37];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L28_002'; gene = 'uninj'; zlims = [30,32];
# filePath = '20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L28_004'; gene = 'uninj'; zlims = [29,31];

#injected files
filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L25_002'; gene = 'nr2e3F0'; zlims = [25,27]
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L25_004'; gene = 'nr2e3F0'; zlims = [26,28];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L26_002'; gene = 'nr2e3F0'; zlims = [23,25];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L26_004'; gene = 'nr2e3F0'; zlims = [43,45];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L27_002'; gene = 'nr2e3F0'; zlims = [23,25];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L27_004'; gene = 'nr2e3F0'; zlims = [29,31]
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L28_002'; gene = 'nr2e3F0'; zlims = [28,30]; 
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L28_004'; gene = 'nr2e3F0'; zlims = [37,39]
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L29_002'; gene = 'nr2e3F0'; zlims = [28,30]; #did not save 001 mip
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L29_004'; gene = 'nr2e3F0'; zlims = [35,37];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L30_002'; gene = 'nr2e3F0'; zlims = [31,33];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L30_002bis'; gene = 'nr2e3F0'; zlims = [29,31]; #no whole eye mip
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L30_004'; gene = 'nr2e3F0'; zlims = [27,29];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L32_002'; gene = 'nr2e3F0'; zlims = [25,27]
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L32_004'; gene = 'nr2e3F0'; zlims = [28,30];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L33_002'; gene = 'nr2e3F0'; zlims = [22,24]
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L33_004'; gene = 'nr2e3F0'; zlims = [34,36];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L34_002'; gene = 'nr2e3F0'; zlims = [38,40];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L34_004'; gene = 'nr2e3F0'; zlims = [39,41]; #exclude optic nerve included
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L35_002'; gene = 'nr2e3F0';zlims = [26,28]; 
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L36_002'; gene = 'nr2e3F0'; zlims = [28,30]; 
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L36_004'; gene = 'nr2e3F0'; zlims = [25,27];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L37_002'; gene = 'nr2e3F0'; zlims = [30,32];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L37_002bis'; gene = 'nr2e3F0'; zlims = [33,35];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L37_004'; gene = 'nr2e3F0'; zlims = [25,27]; 
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L38_002'; gene = 'nr2e3F0'; zlims = [36,38]; 
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L38_004'; gene = 'nr2e3F0'; zlims = [37,39]; #no whole eye mip
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L39_002'; gene = 'nr2e3F0'; zlims = [38,40];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L40_002'; gene = 'nr2e3F0'; zlims = [25,27]; 
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L40_004'; gene = 'nr2e3F0'; zlims = [24,26] #need to reload z stack to dot all uv cones
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L41_002'; gene = 'nr2e3F0'; zlims = [17,19];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L41_004'; gene = 'nr2e3F0'; zlims = [29,31];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L42_002'; gene = 'nr2e3F0'; zlims = [25,27];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L42_004'; gene = 'nr2e3F0'; zlims = [27,29]
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L43_002'; gene = 'nr2e3F0'; zlims = [35,37]
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L43_002_b'; gene = 'nr2e3F0'; zlims = [35,37]; #no map of whole retina
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L43_004'; gene = 'nr2e3F0'; zlims = [26,28]; 
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L44_002'; gene = 'nr2e3F0'; zlims = [30,32]; #no mip taken
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L44_004'; gene = 'nr2e3F0'; zlims = [29,31]; #no whole mip
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L45_002'; gene = 'nr2e3F0'; zlims = [22,24]; 
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L45_004'; gene = 'nr2e3F0'; zlims = [19,21];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L46_002'; gene = 'nr2e3F0';zlims = [21,23];  
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L46_004'; gene = 'nr2e3F0'; zlims = [28,30];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L47_002'; gene = 'nr2e3F0'; zlims = [24,26]; 
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L48_002'; gene = 'nr2e3F0'; zlims = [27,29]; #no whole eye mip taken
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L49_002'; gene = 'nr2e3F0'; zlims = [21,23];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L49_004'; gene = 'nr2e3F0'; zlims = [21,23] #No L49 003 mip
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L50_002'; gene = 'nr2e3F0'; zlims = [20,22];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L50_004'; gene = 'nr2e3F0'; zlims = [27,29]; #No L509 003 mip
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L51_002'; gene = 'nr2e3F0'; zlims = [21,24]; #4 rods outside central retina
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L51_004'; gene = 'nr2e3F0'; zlims = [17,19];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L52_002'; gene = 'nr2e3F0'; zlims = [25,27];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L52_004'; gene = 'nr2e3F0'; zlims = [29,31]
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L56_002'; gene = 'nr2e3F0'; zlims = [24,36];
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L56_002b'; gene = 'nr2e3F0'; zlims = [23,25]; #same as above file
# filePath = '20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L56_004'; gene = 'nr2e3F0'; zlims = [26,28];

# define subdirectory for analysis
dirOut = dirAnalysis + gene + '/' + filePath + '/'
print('Viewer cleared...')

# clear key binds
@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    ...
@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    ...
@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    ...

# load mips
mipO = 0.65
viewer.open(dirOut + layerNames[0] + "_mip.tiff", plugin='napari', colormap = 'green', blending='additive', opacity=mipO);
viewer.open(dirOut + layerNames[1] + "_mip.tiff", plugin='napari', colormap = 'magenta', blending='additive', opacity=mipO);
# viewer.open(dirOut + layerNames[2] + "_mip.tiff", plugin='napari', colormap = 'bop orange', blending='additive', opacity=mipO);

# load segmentation
if (os.path.isfile(dirOut + layerNames[0] + "_seg_curated.tiff")):
    print('Loaded previously curated segmentation for {0}'.format(layerNames[0]))
    viewer.open(dirOut + layerNames[0] + "_seg_curated.tiff", name=layerNames[0] + '_seg', plugin='napari',blending='additive');
else:
    segData = np.load(dirOut + layerNames[0] + "_mip" + "_seg.npy", allow_pickle=True).item()
    viewer.add_labels(segData['masks'], name= layerNames[0] + '_seg',blending='additive');
viewer.layers[layerNames[0] + '_seg'].preserve_labels = True;

if (os.path.isfile(dirOut + layerNames[1] + "_seg_curated.tiff")):
    print('Loaded previously curated segmentation for {0}'.format(layerNames[1]))
    viewer.open(dirOut + layerNames[1] + "_seg_curated.tiff", name=layerNames[1] + '_seg', plugin='napari',blending='additive');
else:
    segData = np.load(dirOut + layerNames[1] + "_mip" + "_seg.npy", allow_pickle=True).item()
    viewer.add_labels(segData['masks'], name= layerNames[1] + '_seg',blending='additive');
viewer.layers[layerNames[1] + '_seg'].preserve_labels = True;


viewer.layers[layerNames[0] + '_mip'].visible = False
# viewer.layers[layerNames[3] + '_mip'].visible = False
viewer.layers[layerNames[0] + '_seg'].visible = False

print('Loaded  ' + filePath + ' !\n Now fix it and save it!')

#define useful keyboard shortcuts

@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
   lname = layerNames[1] + '_seg'
   tempd = viewer.layers[lname].data
   tempd[tempd == viewer.layers[lname].selected_label]=0
   viewer.layers[lname].data = tempd
   print('Cut!')
print('removeLabel on {0}_seg ("Shift-X")'.format(layerNames[1]))

@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
   lname = layerNames[1] + '_seg'
   if (viewer.layers[lname].preserve_labels == True):
      viewer.layers[lname].preserve_labels = False
   elif (viewer.layers[lname].preserve_labels == False):
      viewer.layers[lname].preserve_labels = True
print('toggle {0}_seg preserve_labels ("k")'.format(layerNames[1]))
        
@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
   lname = layerNames[1] + '_mip'
   if (viewer.layers[lname].visible == True):
      viewer.layers[lname].visible = False
   elif (viewer.layers[lname].visible == False):
      viewer.layers[lname].visible = True
print('toggle {0}_mip visibility ("B")'.format(layerNames[1]))

@viewer.bind_key('n', overwrite=True)
def toggle_sel(viewer):
   lname = layerNames[0] + '_mip'
   if (viewer.layers[lname].visible == True):
      viewer.layers[lname].visible = False
   elif (viewer.layers[lname].visible == False):
      viewer.layers[lname].visible = True
print('toggle {0}_mip visibility ("N")'.format(layerNames[0]))

@viewer.bind_key('+', overwrite=True)
def new_label(viewer):
   """Set the currently selected label to the largest used label plus one."""
   lname = layerNames[1] + '_seg'
   viewer.layers[lname].selected_label = viewer.layers[lname].data.max() + 1
print('Add new label in {0}_seg visibility ("+"|"M")'.format(layerNames[1]))

Viewer cleared...


FileNotFoundError: [Errno 2] No such file or directory: 'G:/Shared drives/vldImaging/Analysis/F3tbx2bF0nr2e3_Immunos/xOG_sws1Cy3_rhoCy5/nr2e3F0/20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L25_002/N_mip_seg.npy'

In [147]:
layernames = ['R']

In [37]:
# if needed, run this cell to reload whole z-stack
print('Loading data...')
nMIP = len(viewer.layers)
viewer.open((dirData+filePath + '.nd2'), plugin='napari-nikon-nd2')
viewer.layers.remove(viewer.layers[len(viewer.layers)-1]) # remove transmitted detector image
viewer.layers.select_next();
if len(viewer.layers)==2+nMIP: #no DAPI
   viewer.layers[nMIP+0].colormap = 'green'
   viewer.layers[nMIP+0].name = layerNames[0]
   viewer.layers[nMIP+1].colormap = 'magenta'
   viewer.layers[nMIP+1].name = layerNames[1]
elif len(viewer.layers)==3+nMIP: #DAPI
   viewer.layers[nMIP+0].colormap = 'gray'
   viewer.layers[nMIP+0].name = 'N'
   viewer.layers[nMIP+0].opacity = 0.99
   viewer.layers[nMIP+1].colormap = 'green'
   viewer.layers[nMIP+1].name = layerNames[0]
   viewer.layers[nMIP+2].colormap = 'magenta'
   viewer.layers[nMIP+2].name = layerNames[1]
elif len(viewer.layers)==4+nMIP: #DAPI
   viewer.layers[nMIP+0].colormap = 'gray'
   viewer.layers[nMIP+0].name = 'N'
   viewer.layers[nMIP+0].opacity = 0.99
   viewer.layers[nMIP+1].colormap = 'green'
   viewer.layers[nMIP+1].name = layerNames[0]
   viewer.layers[nMIP+2].colormap = 'magenta'
   viewer.layers[nMIP+2].name = layerNames[1]
   viewer.layers[nMIP+3].colormap = 'bop orange'
   viewer.layers[nMIP+3].name = layerNames[2]

print('Images loaded!')

Loading data...


KeyboardInterrupt: 

#### resave first curated segmentation after napari-ing around

In [13]:
layerNames

['R', 'U', 'Rho']

In [74]:
#Saving only r segmentation 
lname = layerNames[0] + '_seg'
l = viewer.layers[lname]; l.save(dirOut + l.name + '_curated.tiff')
print('Done with ' + lname + ' for ' + filePath)

Done with R_seg for 20220128_tbx2bF3_uninj_xOG_sws1Cy3_rhoCy5_L21_002


In [207]:

viewer.layers[layerNames[0] + '_mip'].visible = True
viewer.layers[layerNames[0] + '_seg'].visible = True
viewer.layers[layerNames[0] + '_seg'].contour = 0
viewer.layers[layerNames[1] + '_mip'].visible = False
viewer.layers[layerNames[1] + '_seg'].visible = False

@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
   lname = layerNames[0] + '_seg'
   tempd = viewer.layers[lname].data
   tempd[tempd == viewer.layers[lname].selected_label]=0
   viewer.layers[lname].data = tempd
   print('Cut!')
print('removeLabel on {0}_seg ("Shift-X")'.format(layerNames[0]))

@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
   lname = layerNames[1] + '_mip'
   if (viewer.layers[lname].visible == True):
      viewer.layers[lname].visible = False
   elif (viewer.layers[lname].visible == False):
      viewer.layers[lname].visible = True
print('toggle {0}_mip visibility ("B")'.format(layerNames[1]))

@viewer.bind_key('n', overwrite=True)
def toggle_sel(viewer):
   lname = layerNames[0] + '_mip'
   if (viewer.layers[lname].visible == True):
      viewer.layers[lname].visible = False
   elif (viewer.layers[lname].visible == False):
      viewer.layers[lname].visible = True
print('toggle {0}_mip visibility ("N")'.format(layerNames[0]))


@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
   lname = layerNames[0] + '_seg'
   if (viewer.layers[lname].preserve_labels == True):
      viewer.layers[lname].preserve_labels = False
   elif (viewer.layers[lname].preserve_labels == False):
      viewer.layers[lname].preserve_labels = True
print('toggle {0}_seg preserve_labels ("k")'.format(layerNames[0]))

@viewer.bind_key('+', overwrite=True)
def new_label(viewer):
   """Set the currently selected label to the largest used label plus one."""
   lname = layerNames[0] + '_seg'
   viewer.layers[lname].selected_label = viewer.layers[lname].data.max() + 1
print('Add new label in {0}_seg visibility ("+"|"M")'.format(layerNames[0]))

lname = layerNames[1] + '_seg'
l = viewer.layers[lname]; l.save(dirOut + l.name + '_curated.tiff')
print('Done with ' + lname + ' for ' + filePath)

removeLabel on R_seg ("Shift-X")
toggle U_mip visibility ("B")
toggle R_mip visibility ("N")
toggle R_seg preserve_labels ("k")
Add new label in R_seg visibility ("+"|"M")
Done with U_seg for 20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L56_004


#### resave second curated segmentation after napari-ing around

In [208]:
lname = layerNames[0] + '_seg'
l = viewer.layers[lname]; l.save(dirOut + l.name + '_curated.tiff')
print('Done with ' + lname + ' for ' + filePath)

Done with R_seg for 20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L56_004


### Reload curated segmentations after saving for final review

In [21]:
# remove all except mips

# viewer.layers.remove(viewer.layers[layerNames[1] + '_seg'])
# viewer.layers.remove(viewer.layers[layerNames[0] + '_seg'])

# viewer.open(dirOut + layerNames[1] + "_seg_curated.tiff", name=layerNames[1] + '_seg', plugin='napari',blending='additive');
# viewer.layers['U_seg'].contour = 5
# viewer.layers[layerNames[1] + '_mip'].visible = True
# viewer.open(dirOut + layerNames[0] + '_seg_curated.tiff', name=layerNames[0] + '_seg', plugin='napari',blending='additive');

print(filePath)
# n0 = len(np.unique(viewer.layers[layerNames[1] + '_seg'].data))-1
# print("Image has {0} {1}".format(n0, layerNames[1]))
# n1 = len(np.unique(viewer.layers[layerNames[0] + '_seg'].data))-1
# print("Image has {0} {1}".format(n1,layerNames[0]))
n2 = len(np.unique(viewer.layers[layerNames[2] + '_seg'].data))-1
print("Image has {0} {1}".format(n1,layerNames[2]))


20220128_tbx2bF3_nr2e3F0_xOG_sws1Cy3_rhoCy5_L25_002


KeyError: "'U_seg' is not in list"

### [Go back and run next one &uarr;](#manualCuration)

In [55]:
layerNames[1]

'U'

In [55]:
layerNames[1]

'U'